In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user cc2742@cornell.edu — sort-by dateRun

!kaggle competitions download -c font-recognition

!unzip -q train_data.csv.zip -d .
!unzip -q train_labels.csv.zip -d .
!unzip -q test_data.csv.zip -d .
!ls

Saving kaggle.json to kaggle.json
kaggle.json
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user cc2742@cornell.edu — sort-by dateRun
  0% 0.00/6.05M [00:00<?, ?B/s]
100% 6.05M/6.05M [00:00<00:00, 55.7MB/s]
  0% 0.00/332k [00:00<?, ?B/s]
100% 332k/332k [00:00<00:00, 102MB/s]
 67% 9.00M/13.4M [00:00<00:00, 51.1MB/s]
100% 13.4M/13.4M [00:00<00:00, 65.6MB/s]
  0% 0.00/429k [00:00<?, ?B/s]
100% 429k/429k [00:00<00:00, 138MB/s]
unzip:  cannot find or open train_labels.csv.zip, train_labels.csv.zip.zip or train_labels.csv.zip.ZIP.
kaggle.json  sample_submission.csv  test_data.csv.zip  train_data.csv.zip
sample_data  test_data.csv	    train_data.csv     train_labels.csv


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, ReLU
from keras import backend as K
from keras.layers.merge import concatenate

batch_size = 128
num_classes = 6
epochs = 60

# input image dimensions
img_rows, img_cols = 20, 20


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
# Read training and test data files
train_label = pd.read_csv("/content/train_labels.csv").values
train_data = pd.read_csv("/content/train_data.csv").values
test_data  = pd.read_csv("/content/test_data.csv").values
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)


(65000, 407)
(65000, 1)
(29221, 407)


In [ ]:
output_labels = np.unique(train_label)
output_size = output_labels.shape[0]
print('output size:', output_size)
output_labels

output size: 6


array(['ARIAL', 'CALIBRI', 'CAMBRIA', 'SERIF', 'TAHOMA', 'TIMES'],
      dtype=object)

In [ ]:
print(train_label)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(np.ravel(train_label))
y_train = np.reshape(y_train, (train_label.shape[0], 1))
print(y_train)
print(y_train.shape)


[['ARIAL']
 ['TIMES']
 ['TIMES']
 ...
 ['SERIF']
 ['TAHOMA']
 ['TAHOMA']]
[[0]
 [5]
 [5]
 ...
 [3]
 [4]
 [4]]
(65000, 1)


In [ ]:
f1_unis = np.unique(train_data[:, 0])
f1_size = f1_unis.shape[0]
f1 = train_data[:, 0]
print('Number of unicodes in training data: ',f1_unis.shape)
#print(f1_unis)

f2_unis = np.unique(test_data[:, 0])
f2_size = f2_unis.shape[0]
f2 = test_data[:, 0]
print('Number of unicodes in test data: ', f2_unis.shape)
#print(f2_unis)

Number of unicodes in training data:  (4225,)
Number of unicodes in test data:  (3773,)


In [ ]:
# One hot encoder
f3 = np.concatenate((train_data[:, 0], test_data[:,0]))
f3_unis = np.unique(f3.reshape(f3.shape[0], 1))
print('Number of unicodes in both training and test data: ', f3_unis.shape)
ohe = preprocessing.OneHotEncoder()
oheout = ohe.fit_transform(f3.reshape(f3.shape[0], 1))
print('One hot encode, shape of first feature: ', oheout.shape)

# Preprocessing MinMaxScaler
f4 = np.concatenate((train_data[:, 1:7], test_data[:,1:7]))
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
mmout = scaler.fit_transform(f4)
print('MinMaxScaling, shape of features:', mmout.shape)

Number of unicodes in both training and test data:  (4541,)
One hot encode, shape of first feature:  (94221, 4541)
MinMaxScaling, shape of features: (94221, 6)


In [ ]:
print(mmout.shape)
mmout
# attr_train = train_data[:, :7]
# attr_test = test_data[:, :7]
# print(attr_train.shape[0], 'train samples')
# print(attr_test.shape[0], 'test samples')

(94221, 6)


array([[0.        , 1.        , 0.35353535, 0.24418605, 0.55      ,
        0.2992126 ],
       [0.        , 0.        , 0.48484848, 0.26744186, 0.3875    ,
        0.18897638],
       [0.        , 0.        , 0.33333333, 0.18604651, 0.1375    ,
        0.05511811],
       ...,
       [0.        , 0.        , 0.        , 0.        , 0.1625    ,
        0.16535433],
       [0.        , 0.        , 0.60606061, 0.20930233, 0.3125    ,
        0.25984252],
       [0.        , 0.        , 0.        , 0.        , 0.125     ,
        0.06299213]])

In [ ]:
import scipy

num_train = train_data.shape[0]
num_test = test_data.shape[0]
print('Array type after transform: ',type(mmout[:num_train, :]), type(b))
print(mmout[:num_train, :].shape, oheout[:num_train,:].shape)
a = mmout[:num_train, :]
b = scipy.sparse.csr_matrix.toarray(oheout[:num_train,:])
print('Array type after transform: ',type(a), type(b))
attr_train2 = np.concatenate((a,b),axis=1) 

a = mmout[num_train:, :]
b = scipy.sparse.csr_matrix.toarray(oheout[num_train:, :])
# attr_train2 = np.concatenate((train_data[:, 1:7], oheout[:train_data.shape[0],:]), axis=0)
attr_test2 = np.concatenate((a, b), axis=1)
print(attr_train2.shape, 'train samples')
print(attr_test2.shape, 'test samples')

Array type after transform:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(65000, 6) (65000, 4541)
Array type after transform:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(65000, 4547) train samples
(29221, 4547) test samples


In [ ]:
print(attr_test2)
print(attr_train2)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.33333333 ... 0.         0.         0.        ]
 [1.         1.         0.32323232 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.60606061 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0.         1.         0.35353535 ... 0.         0.         0.        ]
 [0.         0.         0.48484848 ... 0.         0.         0.        ]
 [0.         0.         0.33333333 ... 0.         0.         0.        ]
 ...
 [1.         0.         0.28282828 ... 0.         0.         0.        ]
 [1.         0.         0.49494949 ... 0.         0.         0.        ]
 [0.         0.         0.42424242 ... 0.         0.         0.        ]]


In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

In [ ]:
# Reshape and normalize training data
x_train = train_data[:, 7:]
print(x_train.shape)
x_test = test_data[:, 7:]
print(x_test.shape)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(65000, 400)
(29221, 400)
x_train shape: (65000, 20, 20, 1)
65000 train samples
29221 test samples


In [ ]:
print(x_train.shape)
print(y_train.shape)

(65000, 20, 20, 1)
(65000, 6)


In [ ]:


input1 = keras.Input(shape=input_shape)
x1 = Conv2D(32, kernel_size=(3, 3), padding="same",activation='relu', input_shape=input_shape)(input1)
x1 = BatchNormalization(momentum=0.1, epsilon=1e-05)(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x1 = Conv2D(64, (3, 3), activation='relu', padding="same")(x1)
x1 = BatchNormalization(momentum=0.1, epsilon=1e-05)(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x1 = Conv2D(128, (3, 3), padding="same", activation='relu')(x1)
x1 = Dropout(0.25)(x1)
x1 = Flatten()(x1)
x1 = Dense(128, activation='relu')(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(32, activation='relu')(x1)
x1 = Dense(16, activation='relu')(x1)

output1 = keras.Model(input1, x1)

# the second branch opreates on the second input
input2 = keras.Input(shape=(4547,))
x2 = Dense(128, activation="relu")(input2)
x2 = Dense(64, activation="relu")(x2)
x2 = Dense(32, activation="relu")(x2)
x2 = Dense(16, activation="relu")(x2)
#y = keras.Model(inputs=inputB, outputs=y)
output2 = keras.Model(input2, x2)

# combine the output of the two branches
combined = concatenate([x1, x1, x2])

# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(16, activation="relu")(combined)
z = Dense(16, activation="relu")(z)
z = Dense(16, activation="relu")(z)
z = Dense(num_classes, activation="softmax")(z)

# our model will accept the inputs of the two branches and
# then output a single value
model = keras.Model(inputs=[input1, input2], outputs=z)


opt = keras.optimizers.Adam(learning_rate=0.002)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit([x_train, attr_train2], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, 
          validation_split=0.1)


Epoch 1/60
458/458 [==============================] - 69s 151ms/step - loss: 1.3817 - accuracy: 0.4141 - val_loss: 1.1556 - val_accuracy: 0.5195
Epoch 2/60
458/458 [==============================] - 75s 165ms/step - loss: 1.0347 - accuracy: 0.5654 - val_loss: 1.0790 - val_accuracy: 0.5535
Epoch 3/60
458/458 [==============================] - 66s 145ms/step - loss: 0.8410 - accuracy: 0.6501 - val_loss: 0.8447 - val_accuracy: 0.6495
Epoch 4/60
458/458 [==============================] - 66s 143ms/step - loss: 0.7143 - accuracy: 0.6978 - val_loss: 0.8419 - val_accuracy: 0.6508
Epoch 5/60
458/458 [==============================] - 66s 144ms/step - loss: 0.6352 - accuracy: 0.7321 - val_loss: 0.7448 - val_accuracy: 0.6923
Epoch 6/60
458/458 [==============================] - 70s 153ms/step - loss: 0.5847 - accuracy: 0.7515 - val_loss: 0.7170 - val_accuracy: 0.7071
Epoch 7/60
458/458 [==============================] - 69s 151ms/step - loss: 0.5324 - accuracy: 0.7752 - val_loss: 0.6861 - val_ac

KeyboardInterrupt: ignored

In [ ]:
Y_test = model.predict([x_test, attr_test2])

In [ ]:
print(Y_test.shape)
print(Y_test)
Y_predict = np.argmax(Y_test, axis=1)
print(Y_predict)

Y_p = le.inverse_transform(Y_predict)

print(type(Y_p.tolist()[0]))


In [ ]:
import csv
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f, dialect='excel')
    writer.writerow(["ID", "Font"])
    for ID, prediction in enumerate(Y_p, 1):
      writer.writerow([ID, prediction])

In [ ]:
import csv
RESULTS = [
    ['apple','cherry','orange','pineapple','strawberry']
]
with open('output.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerows(RESULTS)